<div class="alert alert-block alert-info">
<h3><b> Final Project </b></h3>

**Class:** DATA 230 - Sec 22 - Fall 2024

**Team 5:** Cheng-Huan Yu, Chun-Chieh Kuo, Khac Minh Dai Vo, Lam Tran

**Date:** December 4th, 2024

## Calculate the difference for Health Metrics
This code use to generate the difference in the health metrics that used to plot the bar chart in power bi 
- Cardiavascular
- Liver
- Metabolic

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("smoking.csv")

## Data processing
- covert the smoking status "0" to N, "Y" to 1

In [3]:
df['smoking'] = df['smoking'].replace({0: 'N', 1: 'Y'})

In [4]:
df.columns

Index(['ID', 'gender', 'age', 'height(cm)', 'weight(kg)', 'waist(cm)',
       'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)',
       'systolic', 'relaxation', 'fasting blood sugar', 'Cholesterol',
       'triglyceride', 'HDL', 'LDL', 'hemoglobin', 'Urine protein',
       'serum creatinine', 'AST', 'ALT', 'Gtp', 'oral', 'dental caries',
       'tartar', 'smoking'],
      dtype='object')

## Add column BMI

In [5]:
# Step 1: Calculate BMI
df["height(m)"] = df["height(cm)"] / 100  # Convert height to meters
df["BMI"] = df["weight(kg)"] / (df["height(m)"] ** 2)

# Step 2: Categorize BMI
def categorize_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 24.9:
        return "Normal"
    elif 25 <= bmi < 29.9:
        return "Overweight"
    else:
        return "Obese"

df["BMI Category"] = df["BMI"].apply(categorize_bmi)

# Step 3: Aggregate Data by BMI Category and Gender
metrics = [
    "systolic",
    "relaxation",
    "fasting blood sugar",
    "Cholesterol",
    "triglyceride",
    "LDL",
    "hemoglobin",
    "Urine protein",
    "AST",
    "ALT",
    "Gtp",
]

## Calculate averages for all of features group by (Category, Gender, Smoking)

In [6]:
# Group by BMI Category and Gender, then calculate averages
aggregated_data = (
    df.groupby(["BMI Category", "gender", "smoking"])[metrics]
    .mean()
    .reset_index()
)

# Display the aggregated data
print(aggregated_data)

   BMI Category gender smoking    systolic  relaxation  fasting blood sugar  \
0        Normal      F       N  116.568930   72.534818            94.753686   
1        Normal      F       Y  114.065421   71.502804            97.396262   
2        Normal      M       N  121.075319   75.366670            97.294847   
3        Normal      M       Y  120.972780   76.103419           100.124988   
4         Obese      F       N  124.444976   77.320043           102.402977   
5         Obese      F       Y  123.021053   77.768421           101.210526   
6         Obese      M       N  130.623323   82.401445           105.995872   
7         Obese      M       Y  129.866909   82.490182           107.940364   
8    Overweight      F       N  124.036903   76.974968           100.723355   
9    Overweight      F       Y  121.403727   76.565217           107.118012   
10   Overweight      M       N  125.966742   78.749782           101.241860   
11   Overweight      M       Y  125.432695   79.1003

## Percentage difference of "Overweight" and "Obese"

In [7]:
aggregated_data_obese_overweight = aggregated_data[
    (aggregated_data["BMI Category"].values == "Overweight") | 
    (aggregated_data["BMI Category"].values == "Obese")
]


In [8]:
aggregated_data_obese_overweight

,BMI Category,gender,smoking,systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,LDL,hemoglobin,Urine protein,AST,ALT,Gtp
4,Obese,F,N,124.444976,77.320043,102.402977,204.543328,122.745348,121.463583,13.337161,1.093036,25.315789,24.614567,30.085061
5,Obese,F,Y,123.021053,77.768421,101.210526,209.452632,156.315789,117.747368,13.692632,1.115789,23.400000,23.947368,38.536842
6,Obese,M,N,130.623323,82.401445,105.995872,199.323013,161.844169,118.172343,15.659443,1.123839,35.220846,51.644995,57.108359
7,Obese,M,Y,129.866909,82.490182,107.940364,201.752727,183.324364,118.107636,15.904000,1.191273,35.642182,51.503273,75.938182
8,Overweight,F,N,124.036903,76.974968,100.723355,205.922065,125.568258,122.487742,13.330581,1.092387,25.545548,24.457548,28.908129
9,Overweight,F,Y,121.403727,76.565217,107.118012,205.521739,160.968944,115.316770,13.683851,1.099379,24.229814,23.763975,44.881988
10,Overweight,M,N,125.966742,78.749782,101.241860,197.146962,141.008706,118.502351,15.451175,1.103604,28.980672,34.906147,46.768762
11,Overweight,M,Y,125.432695,79.100394,104.344128,199.715573,171.285976,116.717967,15.688341,1.093495,28.538158,35.755562,61.100957


In [9]:
# Group by gender and BMI Category
grouped = aggregated_data_obese_overweight.groupby(["gender", "BMI Category", "smoking"]).mean()

# Separate data for Obese and Overweight by gender
obese = grouped.xs("Obese", level="BMI Category")
overweight = grouped.xs("Overweight", level="BMI Category")

# Calculate percentage difference for each gender
percentage_diff = ((overweight - obese) / obese) * 100

# Combine results into a DataFrame
results = percentage_diff.reset_index()

# Rename columns for clarity
results.rename(columns={
    "gender": "Gender",
    "systolic": "Systolic (%) Diff",
    "relaxation": "Relaxation (%) Diff",
    "fasting blood sugar": "Fasting Blood Sugar (%) Diff",
    "Cholesterol": "Cholesterol (%) Diff",
    "triglyceride": "Triglyceride (%) Diff",
    "LDL": "LDL (%) Diff",
    "hemoglobin": "Hemoglobin (%) Diff",
    "AST": "AST (%) Diff",
    "ALT": "ALT (%) Diff",
    "Gtp": "GTP (%) Diff"
}, inplace=True)

# Print the results
print(results)
results.to_csv("percentage_diff_obese_overweight.csv")

  Gender smoking  Systolic (%) Diff  Relaxation (%) Diff  \
0      F       N          -0.327914            -0.446294   
1      F       Y          -1.314674            -1.547162   
2      M       N          -3.564892            -4.431551   
3      M       Y          -3.414430            -4.109322   

   Fasting Blood Sugar (%) Diff  Cholesterol (%) Diff  Triglyceride (%) Diff  \
0                     -1.640208              0.674056               2.299810   
1                      5.836830             -1.876745               2.976766   
2                     -4.485092             -1.091721             -12.873780   
3                     -3.331687             -1.009728              -6.566715   

   LDL (%) Diff  Hemoglobin (%) Diff  Urine protein  AST (%) Diff  \
0      0.843182            -0.049339      -0.059332      0.907572   
1     -2.064248            -0.064127      -1.470761      3.546212   
2      0.279260            -1.329980      -1.800491    -17.717275   
3     -1.176613       

## Percentage different of "Normal" and "Underweight"

In [10]:
aggregated_data_normal_underweight = aggregated_data[
    (aggregated_data["BMI Category"].values == "Underweight") | 
    (aggregated_data["BMI Category"].values == "Normal")
]

In [11]:
aggregated_data_normal_underweight

,BMI Category,gender,smoking,systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,LDL,hemoglobin,Urine protein,AST,ALT,Gtp
0,Normal,F,N,116.568930,72.534818,94.753686,199.598965,95.050502,116.216280,13.094111,1.072302,23.138645,18.567754,21.529407
1,Normal,F,Y,114.065421,71.502804,97.396262,196.379439,110.891589,109.280374,13.348972,1.074766,23.413084,18.719626,31.547664
2,Normal,M,N,121.075319,75.366670,97.294847,190.983313,112.108129,112.695343,15.277235,1.074304,25.277596,25.377156,33.577743
3,Normal,M,Y,120.972780,76.103419,100.124988,192.440897,136.247904,110.142884,15.402100,1.084770,26.419987,26.513799,50.770557
12,Underweight,F,N,111.551948,69.893939,90.946970,194.066017,75.857143,107.672078,12.962771,1.079004,21.432900,15.306277,18.451299
13,Underweight,F,Y,108.514706,71.088235,91.705882,190.191176,98.750000,96.779412,13.533824,1.029412,22.029412,17.852941,42.632353
14,Underweight,M,N,116.723214,72.500000,94.535714,176.553571,82.441964,96.138393,14.849107,1.071429,24.955357,18.852679,28.535714
15,Underweight,M,Y,116.756972,73.378486,99.039841,179.045817,106.252988,92.428287,15.055378,1.085657,27.942231,20.549801,57.059761


In [12]:
# Group by gender and BMI Category
grouped = aggregated_data_normal_underweight.groupby(["gender", "BMI Category", "smoking"]).mean()

# Separate data for Normal and Underweight by gender
normal = grouped.xs("Normal", level="BMI Category")
underweight = grouped.xs("Underweight", level="BMI Category")

# Calculate percentage difference for each gender
percentage_diff = ((underweight - normal) / normal) * 100

# Combine results into a DataFrame
results = percentage_diff.reset_index()

# Rename columns for clarity
results.rename(columns={
    "gender": "Gender",
    "systolic": "Systolic (%) Diff",
    "relaxation": "Relaxation (%) Diff",
    "fasting blood sugar": "Fasting Blood Sugar (%) Diff",
    "Cholesterol": "Cholesterol (%) Diff",
    "triglyceride": "Triglyceride (%) Diff",
    "LDL": "LDL (%) Diff",
    "hemoglobin": "Hemoglobin (%) Diff",
    "AST": "AST (%) Diff",
    "ALT": "ALT (%) Diff",
    "Gtp": "GTP (%) Diff"
}, inplace=True)

# Print the results
print(results)
results.to_csv("percentage_diff_normal_underweight.csv")

  Gender smoking  Systolic (%) Diff  Relaxation (%) Diff  \
0      F       N          -4.303876            -3.640843   
1      F       Y          -4.866255            -0.579793   
2      M       N          -3.594543            -3.803632   
3      M       Y          -3.484922            -3.580566   

   Fasting Blood Sugar (%) Diff  Cholesterol (%) Diff  Triglyceride (%) Diff  \
0                     -4.017486             -2.772032             -20.192801   
1                     -5.842503             -3.151177             -10.949062   
2                     -2.835847             -7.555498             -26.462100   
3                     -1.083793             -6.960620             -22.014956   

   LDL (%) Diff  Hemoglobin (%) Diff  Urine protein  AST (%) Diff  \
0     -7.351984            -1.003048       0.625005     -7.371843   
1    -11.439348             1.384763      -4.219949     -5.909825   
2    -14.691779            -2.802393      -0.267632     -1.274801   
3    -16.083288       